In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
features = dict(np.load("../feature/frequent_feature.npz"))
labels = dict(np.load("../feature/label.npz"))

feature = None
label = None
for key in features:
    if feature is None:
        feature = features[key]
        label = labels[key]
    else:
        feature = np.concatenate((feature, features[key]))
        label = np.concatenate((label, labels[key]))

In [3]:
records = {
    "ann": np.zeros(10),
    "rf": np.zeros(10),
    "svm": np.zeros(10)
}

In [4]:
for k in range(10):
    clfs = {
        "ann": MLPClassifier(hidden_layer_sizes=(128)),
        "rf": RandomForestClassifier(),
        "svm": SVC()
    }
    #shuffle
    perm = np.random.permutation(feature.shape[0])
    feature = feature[perm]
    label = label[perm]
    
    radio = 0.8

    train_index = slice(0, int(radio * feature.shape[0]))
    valid_index = slice(int(radio * feature.shape[0]), feature.shape[0])

    x_train = feature[train_index]
    y_train = label[train_index]
    x_valid = feature[valid_index]
    y_valid = label[valid_index]
    scaler = StandardScaler()
    x_train_norm = scaler.fit_transform(x_train)
    x_valid_norm = scaler.transform(x_valid)
    
    for key, clf in clfs.items():
        clf.fit(x_train_norm, y_train)
        y_valid_predict = clf.predict(x_valid_norm)
        records[key][k] = np.sum(y_valid_predict.reshape([-1, 1]) == y_valid) / len(y_valid)

D:\Software\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
D:\Software\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Software\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

In [5]:
for key, value in records.items():
    print("%s: %f ± %f" % (key, value.mean(), value.std()))

ann: 0.993750 ± 0.001191
rf: 0.973037 ± 0.003938
svm: 0.986018 ± 0.002420


In [6]:
clfs["ann"]

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=128, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [7]:
clfs["svm"]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [8]:
clfs["rf"]

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)